In [12]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [13]:
from langchain.agents import tool
from typing import List, Dict, Annotated

In [14]:
from langchain_teddynote.tools import GoogleNews
from langchain_experimental.utilities import PythonREPL

In [15]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

In [16]:
@tool
def search_keyword(query: str) -> List[Dict[str, str]]:
    """Look up news by keyword"""

    print(f'검색어: {query}')

    news_tool = GoogleNews()

    return news_tool.search_by_keyword(query, k=2)

In [17]:
@tool
def python_repl_tool(
    code: Annotated[str, "The python code to execute to generate your chart."],
):
    """Use this to execute python code. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user."""

    result = ""
    
    try:
        result = PythonREPL().run(code)
    except BaseException as e:
        print(f"Failed to execute. Error: {repr(e)}")
    finally:
        return result

In [18]:
print(f'도구 이름: {search_keyword.name}')
print(f'도구 설명: {search_keyword.description}')

도구 이름: search_keyword
도구 설명: Look up news by keyword


In [19]:
print(f'도구 이름: {python_repl_tool.name}')
print(f'도구 설명: {python_repl_tool.description}')

도구 이름: python_repl_tool
도구 설명: Use this to execute python code. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user.


In [20]:
tools = [search_keyword, python_repl_tool]      # 여러 개의 도구를 한꺼번에 바인딩 위해서 리스트에 넣기     

### Agent 프롬프트

- `chat_history` : 이전 대화 내용을 저장하는 변수 (생략 가능)
- `agent_scratchpad` : 에이전트가 검색하는 과정들이나 내용을 임시로 저장하는 변수
- `input` : 사용자 입력

In [21]:
# 프롬프트는 에이전트에게 모델이 수행할 작업을 설명하는 텍스트를 제공합니다. (도구의 이름과 역할을 입력)
prompt = ChatPromptTemplate.from_messages(
    [
        (                                           
            "system",
            "You are a helpful assistant. "             
            "Make sure to use the `search_news` tool for searching keyword related news."
        ),
        ("placeholder", "{chat_history}"),          # 이전 대화 내용을 넣을 곳을 잡아둔다.
        ("human", "{input}"),                       # 사용자 입력

        # 에이전트가 검색하는 과정들이나 내용 등을 끄적이는 메모장 같은 공간을 플레이스 홀더로 만들어준다
        ("placeholder", "{agent_scratchpad}"),       
    ]
)

### Agent 생성

In [22]:
llm = ChatOpenAI(
    api_key=key, 
    model='gpt-4o-mini', 
    temperature=0
)

agent = create_tool_calling_agent(llm, tools, prompt)       # 에이전트      

### AgentExecutor

AgentExecutor는 도구를 사용하는 에이전트를 실행하는 클래스

- agent : 각 단계에서 계획을 생성하고 행동을 결정하는 agent
- tools : agent 가 사용할 수 있는 도구 목록
- max_iterations : 최대 몇 번까지 반복할지 설정
- max_execution_time : 실행되는데 소요되는 최대 시간(소요 되는 시간이 너무 길지 않도록 제한을 걸때)
- handle_parsing_errors : 오류가 발생했을 때 처리 방법(True, False)

In [23]:
agent_executor = AgentExecutor(
    agent=agent,                    # 각 단계에서 계획을 생성하고 행동을 결정하는 agent
    tools=tools,                    # agent 가 사용할 수 있는 도구 목록
    verbose=True,                   # 중간 단계 결과를 출력
    max_iterations=10,              # 최대 10번 까지 반복
    max_execution_time=10,          # 실행되는데 소요되는 최대 시간
    handle_parsing_errors=True      
)

In [24]:
answer = agent_executor.invoke({'input': '대구 중앙떡볶이에 대한 뉴스를 검색해 주세요.'})



> Entering new AgentExecutor chain...

Invoking: `search_keyword` with `{'query': '대구 중앙떡볶이'}`


검색어: 대구 중앙떡볶이
[{'url': 'https://news.google.com/rss/articles/CBMiaEFVX3lxTE9iUlVjeGpoMmZGWnRyTzRBSnJlbXQ0bzFqalczSkIya3o4TG83TjNOTE5SMS1sbVRRUy1fYkZDeXlKeHZQcldwSTdGUHdFQUZtYTltVlgzbmVmMDhBT1ctZ1VOekN0STJZ?oc=5', 'content': '떡볶이에 진심인, 대구 떡볶이 성지 6 - 대구일보'}, {'url': 'https://news.google.com/rss/articles/CBMiU0FVX3lxTE90TjFnS00xVUNoN3Y3dzlEb2l3cFEtUVkwWlVNekxwaC1Rd0U4dUpTalNYa04teW52b0hjdllBQnpqNDhpSm1aa0lFNXVsMkV2QUFB?oc=5', 'content': "'전현무계획' 대구 떡볶이 4대장 반할만떡·달고떡볶이·윤옥연할매떡볶이·중앙떡볶이→뭉티기X오드레기&서문시장 섞어+냉잔치국수 맛집 탐방 - 네이트 뉴스"}]대구 중앙떡볶이에 대한 최근 뉴스는 다음과 같습니다:

1. **대구 떡볶이 성지 6** - 대구일보에서 보도한 기사로, 대구의 떡볶이에 대한 진심을 다룬 내용입니다. [기사 링크](https://news.google.com/rss/articles/CBMiaEFVX3lxTE9iUlVjeGpoMmZGWnRyTzRBSnJlbXQ0bzFqalczSkIya3o4TG83TjNOTE5SMS1sbVRRUy1fYkZDeXlKeHZQcldwSTdGUHdFQUZtYTltVlgzbmVmMDhBT1ctZ1VOekN0STJZ?oc=5)

2. **'전현무계획' 대구 떡볶이 4대장** - 네이트 뉴스에서 보도된 기사로, 대구의 유명 떡볶이 가게들 중 중앙떡볶이를 포함한 4대장에 대한 탐방 내

In [25]:
answer

{'input': '대구 중앙떡볶이에 대한 뉴스를 검색해 주세요.',
 'output': "대구 중앙떡볶이에 대한 최근 뉴스는 다음과 같습니다:\n\n1. **대구 떡볶이 성지 6** - 대구일보에서 보도한 기사로, 대구의 떡볶이에 대한 진심을 다룬 내용입니다. [기사 링크](https://news.google.com/rss/articles/CBMiaEFVX3lxTE9iUlVjeGpoMmZGWnRyTzRBSnJlbXQ0bzFqalczSkIya3o4TG83TjNOTE5SMS1sbVRRUy1fYkZDeXlKeHZQcldwSTdGUHdFQUZtYTltVlgzbmVmMDhBT1ctZ1VOekN0STJZ?oc=5)\n\n2. **'전현무계획' 대구 떡볶이 4대장** - 네이트 뉴스에서 보도된 기사로, 대구의 유명 떡볶이 가게들 중 중앙떡볶이를 포함한 4대장에 대한 탐방 내용을 다루고 있습니다. [기사 링크](https://news.google.com/rss/articles/CBMiU0FVX3lxTE90TjFnS00xVUNoN3Y3dzlEb2l3cFEtUVkwWlVNekxwaC1Rd0U4dUpTalNYa04teW52b0hjdllBQnpqNDhpSm1aa0lFNXVsMkV2QUFB?oc=5)\n\n더 많은 정보가 필요하시면 말씀해 주세요!"}

In [26]:
print(answer['output'])

대구 중앙떡볶이에 대한 최근 뉴스는 다음과 같습니다:

1. **대구 떡볶이 성지 6** - 대구일보에서 보도한 기사로, 대구의 떡볶이에 대한 진심을 다룬 내용입니다. [기사 링크](https://news.google.com/rss/articles/CBMiaEFVX3lxTE9iUlVjeGpoMmZGWnRyTzRBSnJlbXQ0bzFqalczSkIya3o4TG83TjNOTE5SMS1sbVRRUy1fYkZDeXlKeHZQcldwSTdGUHdFQUZtYTltVlgzbmVmMDhBT1ctZ1VOekN0STJZ?oc=5)

2. **'전현무계획' 대구 떡볶이 4대장** - 네이트 뉴스에서 보도된 기사로, 대구의 유명 떡볶이 가게들 중 중앙떡볶이를 포함한 4대장에 대한 탐방 내용을 다루고 있습니다. [기사 링크](https://news.google.com/rss/articles/CBMiU0FVX3lxTE90TjFnS00xVUNoN3Y3dzlEb2l3cFEtUVkwWlVNekxwaC1Rd0U4dUpTalNYa04teW52b0hjdllBQnpqNDhpSm1aa0lFNXVsMkV2QUFB?oc=5)

더 많은 정보가 필요하시면 말씀해 주세요!
